In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install findspark
!pip install pyspark
!pip install sparknlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 53 kB/s 
     |████████████████████████████████| 199 kB 78.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845514 sha256=3d4c7f81a7cbd246eb16b8005c35c88b22ff7c240bdc5b68a5684576cd2ba142
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 648 kB 33.0 MB/s 


In [ ]:
from sparknlp.pretrained import PretrainedPipeline
import sparknlp
import pyspark.sql.functions as F
from sparknlp.base import *
from sparknlp.annotator import *

In [ ]:
spark = sparknlp.start(gpu=True)
sc = spark.sparkContext

In [ ]:
# comment = spark.read.parquet("/content/drive/MyDrive/data/comments_all")
# submission = spark.read.parquet("/content/drive/MyDrive/data/submissions_all")
# submission_all = submission.withColumn('text', F.concat(F.col('title'),F.lit(' '), F.col('selftext')))

# Dectect Languages for Submission. Only Keep English.

In [ ]:
# submission_lang = language_detect_pipeline.transform(submission_all)
# submission_lang = submission_lang.withColumn("Russian", submission_lang.language[0].metadata.ru)
# submission_lang = submission_lang.withColumn("Ukrainian", submission_lang.language[0].metadata.uk)
# submission_lang = submission_lang.withColumn("English", submission_lang.language[0].metadata.en)
# submission_lang = submission_lang.drop("language", "document", "sentence")
# submission_en = submission_lang.filter(F.col('English')>0.8)
# submission_en.write.mode('overwrite').format("parquet").save("/content/drive/MyDrive/data/submissions_en")

# Dectect Languages for Comments. Only Keep English.

In [ ]:
# comment = comment.withColumnRenamed('body','text')
# comment_lang = language_detect_pipeline.transform(comment)
# comment_lang = comment_lang.withColumn("Russian", comment_lang.language[0].metadata.ru)
# comment_lang = comment_lang.withColumn("Ukrainian", comment_lang.language[0].metadata.uk)
# comment_lang = comment_lang.withColumn("English", comment_lang.language[0].metadata.en)
# comment_lang = comment_lang.drop("language", "document", "sentence")
# comment_en = comment_lang.filter(F.col('English')>0.8)
# comment_en.write.mode('overwrite').format("parquet").save("/content/drive/MyDrive/data/comments_en")

In [ ]:
comment_en = spark.read.parquet("/content/drive/Shareddrives/502_project/data/comments_en")
submission_en = spark.read.parquet("/content/drive/Shareddrives/502_project/data/submissions_en")

# Filter Empty Rows

In [ ]:
remove = ["\[deleted\]","\[removed\]", '\[deleted by user\]']

submission_en = submission_en.withColumn('text', F.trim(F.regexp_replace('text','|'.join(['('+term+')' for term in remove]),'')))
submission_en = submission_en.filter(F.size(F.split(F.col('text'), ' '))>=2)

comment_en = comment_en.withColumn('text', F.trim(F.regexp_replace('text','|'.join(['('+term+')' for term in remove]),'')))
comment_en = comment_en.filter(F.size(F.split(F.col('text'), ' '))>=2)

In [ ]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

documentNormalizer = DocumentNormalizer() \
    .setInputCols("document") \
    .setOutputCol("normalizedDocument") \
    .setAction("clean") \
    .setPatterns(["[^A-Za-z]"])\
    .setReplacement(" ") \
    .setLowercase(True)

tokenizer = Tokenizer() \
  .setInputCols(["normalizedDocument"]) \
  .setOutputCol("token")

stopwords_cleaner = StopWordsCleaner()\
  .setInputCols("token")\
  .setOutputCol("cleanTokens")\

tokenassembler = TokenAssembler()\
    .setInputCols(["normalizedDocument", "cleanTokens"]) \
    .setOutputCol("clean_text_tokens")

pipeline = Pipeline(stages=[documentAssembler,
                            documentNormalizer,
                            tokenizer, 
                            stopwords_cleaner,
                            tokenassembler])
submission_cleaned = pipeline.fit(submission_en).transform(submission_en)
submission_cleaned = submission_cleaned.withColumn("cleaned_text", F.explode(submission_cleaned.clean_text_tokens.result))
submission_cleaned = submission_cleaned.drop("document", "token", "cleanTokens", "clean_text_tokens")
submission_cleaned.write.mode('overwrite').format("parquet").save("/content/drive/Shareddrives/502_project/data/submissions_en_cleaned")

In [ ]:
!wget -q https://raw.githubusercontent.com/******/vocabulary/master/lemmas/AntBNC_lemmas_ver_001.txt

In [ ]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

documentNormalizer = DocumentNormalizer() \
    .setInputCols("document") \
    .setOutputCol("normalizedDocument") \
    .setAction("clean") \
    .setPatterns(["[^A-Za-z]"])\
    .setReplacement(" ") \
    .setLowercase(True)

tokenizer = Tokenizer() \
  .setInputCols(["normalizedDocument"]) \
  .setOutputCol("token")

stopwords_cleaner = StopWordsCleaner()\
  .setInputCols("token")\
  .setOutputCol("cleanTokens")\

stemmer = Stemmer() \
    .setInputCols(["cleanTokens"]) \
    .setOutputCol("stem")

lemmatizer = Lemmatizer() \
    .setInputCols(["stem"]) \
    .setOutputCol("lemma") \
    .setDictionary("./AntBNC_lemmas_ver_001.txt", value_delimiter ="\t", key_delimiter = "->")

tokenassembler = TokenAssembler()\
    .setInputCols(["normalizedDocument", "lemma"]) \
    .setOutputCol("clean_text_tokens")

pipeline = Pipeline(stages=[documentAssembler,
                            documentNormalizer,
                            tokenizer, 
                            stopwords_cleaner,
                            stemmer,
                            lemmatizer,
                            tokenassembler])

submission_cleaned = pipeline.fit(submission_en).transform(submission_en)
submission_cleaned = submission_cleaned.withColumn("cleaned_text", F.explode(submission_cleaned.clean_text_tokens.result))
submission_cleaned = submission_cleaned.drop("document", "token", "cleanTokens", "clean_text_tokens")
submission_cleaned.write.mode('overwrite').format("parquet").save("/content/drive/Shareddrives/502_project/data/submissions_en_lemmatized")

In [ ]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

documentNormalizer = DocumentNormalizer() \
    .setInputCols("document") \
    .setOutputCol("normalizedDocument") \
    .setAction("clean") \
    .setPatterns(["[^A-Za-z]"])\
    .setReplacement(" ") \
    .setLowercase(True)

tokenizer = Tokenizer() \
  .setInputCols(["normalizedDocument"]) \
  .setOutputCol("token")

stopwords_cleaner = StopWordsCleaner()\
  .setInputCols("token")\
  .setOutputCol("cleanTokens")\

tokenassembler = TokenAssembler()\
    .setInputCols(["normalizedDocument", "cleanTokens"]) \
    .setOutputCol("clean_text_tokens")

pipeline = Pipeline(stages=[documentAssembler,
                            documentNormalizer,
                            tokenizer, 
                            stopwords_cleaner,
                            tokenassembler])
comment_cleaned = pipeline.fit(comment_en).transform(comment_en)
comment_cleaned = comment_cleaned.withColumn("cleaned_text", F.explode(comment_cleaned.clean_text_tokens.result))
comment_cleaned = comment_cleaned.drop("document", "token", "cleanTokens", "clean_text_tokens")
comment_cleaned.write.mode('overwrite').format("parquet").save("/content/drive/Shareddrives/502_project/data/comments_en_cleaned")

In [ ]:
comment_en = spark.read.parquet("/content/drive/Shareddrives/502_project/data/comments_en_cleaned")
comment_en.count()

6762807